In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from quick_pp.objects import Project

# Load well from saved file
project = "BEKK_CLEAN"
project_path = rf"data\04_project\{project}.qppp"
project = Project().load(project_path)
project.get_well_names()

In [ ]:
# Load data
well_name = 'BEKK-005'
well = project.get_well(well_name)
well_data = well.data[['WELL_NAME', 'DEPTH', 'ZONES', 'GR', 'RT', 'NPHI', 'RHOB', 'CALI', 'BS']].copy()

# Change the depth unit to meter
well_data['DEPTH'] = well_data['DEPTH'] / 3.2808
well.depth_uom = 'm'

min_depth = well_data.DEPTH.min()
max_depth = well_data.DEPTH.max()
well_data = well_data[(well_data.DEPTH >= min_depth) & (well_data.DEPTH <= max_depth)].copy()

In [ ]:
import pandas as pd

core_df = pd.read_csv(rf"data\04_project\Bekok-5 Sidewall Core.csv")
core_df['DEPTH'] = core_df['Depth (m MD)']
core_df['CPORE'] = core_df['Porosity (%)'] / 100
core_df['CPERM'] = core_df['Perm (mD)']
core_df['CSAT'] = core_df['Sw (%)'] / 100
core_df['DEPTH_FT'] = core_df['DEPTH'] * 3.2808
core_df.sort_values('DEPTH', inplace=True)
well_data = pd.merge_asof(well_data, core_df[['DEPTH', 'CPORE', 'CPERM', 'CSAT']], on='DEPTH', direction='nearest', tolerance=.1524)

# Quick PP Interpretation

In [ ]:
from quick_pp.lithology.sand_silt_clay import SandSiltClay
from quick_pp.porosity import neu_den_xplot_poro, density_porosity, rho_matrix, normalize_volumetric, clay_porosity
from quick_pp.qaqc import badhole_flagging, mask_outside_threshold, neu_den_xplot_hc_correction, den_correction
from quick_pp.saturation import *
from quick_pp.plotter import *
from quick_pp.permeability import *
from quick_pp.ressum import *
from quick_pp.rock_type import rock_typing
from quick_pp.plotter import plotly_log
from quick_pp.utils import zone_flagging, min_max_line

In [ ]:
# Clean up data
well_data = badhole_flagging(well_data)
# well_data = mask_outside_threshold(well_data, fill=True)

# Initialize lithology model
args = {
    'litho_model': 'ssc',
    # 'dry_clay_point': (.3, 2.7),
    'silt_line_angle': 117,
    'wet_clay_point': (0.43, 2.6),
    'sw_water_salinity': 15000,
    'sw_m': 1.9,
    'hc_corr_angle': 50,
    'hc_buffer': 0.01,
    'ressum_cutoffs': dict(
        VSHALE=.5,
        PHIT=0,
        SWT=1
    ),
}
ssc_model = SandSiltClay(**args)
vsand, vsilt, vcld, (nphi_max_line, rhob_max_line) = ssc_model.estimate_lithology(
    nphi=well_data['NPHI'], rhob=well_data['RHOB']
)
args.update(ssc_model.__dict__)
well.update_config(args)  # Save lithology model to well

# Implement hydrocarbon correction
nphihc, rhobhc, hc_flag = neu_den_xplot_hc_correction(
    well_data['NPHI'], well_data['RHOB'], gr=well_data['GR'],
    dry_sand_point=args['dry_sand_point'],
    dry_clay_point=args['dry_clay_point'],
    corr_angle=args['hc_corr_angle'], buffer=args['hc_buffer']
)

# Correct density log
rhob_corr = den_correction(nphihc, well_data['GR'], alpha=0.1)
badhole_flag =  np.where(abs(well_data['RHOB'] - rhob_corr) > 0.2, 1, 0)
well_data['RHOB_CORR'] = rhob_corr = np.where((badhole_flag == 1) & (hc_flag == 0), rhob_corr, rhobhc)

# Estimate lithology
ssc_model = SandSiltClay(**args)
vsand, vsilt, vcld, (nphi_max_line, rhob_max_line) = ssc_model.estimate_lithology(
    nphi=nphihc, rhob=rhob_corr,
)

# Estimate porosity
phit = neu_den_xplot_poro(
    nphihc, rhob_corr, model='ssc', reservoir=True,
    dry_sand_point=args['dry_sand_point'],
    dry_silt_point=args['dry_silt_point'],
    dry_clay_point=args['dry_clay_point'],
)
rho_ma = rho_matrix(vsand, vsilt, vcld)
phid = density_porosity(rhob_corr, rho_ma)

# Normalize lithology
vsand, vsilt, vcld = normalize_volumetric(vsand, vsilt, vcld, phit)

# Calculate vclb: volume of clay bound water and phie
clay_phit = clay_porosity(rhob_corr, args['dry_clay_point'][1])
vclb = vcld * clay_phit
vclw = vcld + vclb
phie = phit - vclb

# Estimate water saturation
tvdss = well_data['DEPTH'] * .8
temp_grad = estimate_temperature_gradient(tvdss, 'metric')
water_salinity = args['sw_water_salinity']
rw_salinity = estimate_rw_temperature_salinity(temp_grad, water_salinity)

b = estimate_b_waxman_smits(temp_grad, rw_salinity)
qv = estimate_qv(vcld, phit, cec_clay=.1)

rw = estimate_rw_from_shale_trend(well_data['RT'], phit, m=1.2)
# rw = estimate_rw_archie(phit, rt_water)
# rw = estimate_rw_waxman_smits(phit, rt_water, B=b, Qv=qv)

# TODO: Implement m estimation/ based on zonation

swt = waxman_smits_saturation(well_data['RT'], rw_salinity, phit, qv, m=args['sw_m'])
# swt_a = archie_saturation(well_data['RT'], rw_salinity, phit, 1, 2, 2)

# Estimate permeability
perm = choo_permeability(vclw, vsilt, phit)
# constant = vclb**1.75
# Swirr = constant / phit
# perm_kc = kozeny_carman_permeability(phit, Swirr)
# perm_t = timur_permeability(phit, Swirr)
# perm_c = coates_permeability(phit, Swirr)
# perm_tx = tixier_permeability(phit, Swirr)

# Estimate rock types
rock_flag = rock_typing(vclw, higher_is_better=False)

# Update ZONES
well_data = zone_flagging(well_data)
zones = well_data.ZONES

# Estimate reservoir summary
ressum_df = calc_reservoir_summary(well_data.DEPTH, vclw, phit, swt, perm, zones, cutoffs=args['ressum_cutoffs'])

# Update data in the project
well_data['NPHI_HC'] = nphihc
well_data['RHOB_HC'] = rhobhc
well_data['HC_FLAG'] = hc_flag
well_data['VSAND'] = vsand
well_data['VSILT'] = vsilt
well_data['VCLB'] = vclb
well_data['VCLD'] = vcld
well_data['VCLW'] = vclw
well_data['PHIT'] = phit
well_data['PHIE'] = phie
well_data['PHID'] = phid
well_data['RW'] = rw
well_data['B'] = b
well_data['Qv'] = qv
well_data['M'] = args['sw_m']
well_data['SWT'] = swt
# well_data['SWE'] = swe
well_data['BVW'] = swt * phie
well_data['PERM'] = perm
# well_data['CPERM'] = perm
well_data['ROCK_FLAG'] = rock_flag
well_data['VHC'] = phit * (1 - swt)
well_data['BADHOLE'] = badhole_flag

In [ ]:
fig, (ax2, ax1) = plt.subplots(nrows=2, figsize=(20, 3), sharex=True)

ax1.plot(well_data.DEPTH, well_data.NPHI, label='NPHI')
ax1.plot(well_data.DEPTH, nphi_max_line, label='NPHI_line')
ax1.set_ylim(.45, -.15)
ax1.legend()

ax2.plot(well_data.DEPTH, well_data.RHOB, label='RHOB')
ax2.plot(well_data.DEPTH, rhob_max_line, label='RHOB_line')
ax2.set_ylim(1.85, 2.85)
ax2.legend()

In [ ]:
copy_args = args.copy()
copy_args.pop('silt_line_angle')
print(copy_args)
neutron_density_xplot(well_data['NPHI'], well_data['RHOB'], **copy_args)

In [ ]:
neutron_density_xplot(nphihc, rhobhc, **copy_args)

In [ ]:
# Debug water saturation
test = well_data.copy()
water_salinity = 15000
m = 1.9

temp_grad = estimate_temperature_gradient(test['DEPTH'], 'metric')
rw_salinity = estimate_rw_temperature_salinity(temp_grad, water_salinity)
rw_archie = estimate_rw_archie(test['PHIT'], test['RT'], m=m)

b_sal = estimate_b_waxman_smits(temp_grad, rw_salinity)
b_archie = estimate_b_waxman_smits(temp_grad, rw_archie)

qv = estimate_qv(vcld, phit, cec_clay=.1)

rw_shale = estimate_rw_from_shale_trend(test['RT'], phit, m=1.2)

swt_sal = waxman_smits_saturation(test['RT'], rw_salinity, test['PHIT'], qv, m=m)
swt_ws = waxman_smits_saturation(test['RT'], rw_shale, test['PHIT'], qv)

# Plotting
fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(nrows=5, sharex=True, figsize=(20, 7))

ax1.plot(test.DEPTH, qv, label='Qv')
ax1.set_frame_on(False)
ax1.set_ylim(0, 10)
ax1.legend()

ax2.plot(test.DEPTH, b_sal, label='b_sal')
ax2.plot(test.DEPTH, b_archie, label='b_archie')
ax2.set_frame_on(False)
ax2.legend()

ax3.plot(test.DEPTH, test['RT'], label='RT')
ax3.set_frame_on(False)
ax3.set_yscale('log')
ax3.legend()

ax4.plot(test.DEPTH, rw_salinity, label='rw_salinity')
ax4.plot(test.DEPTH, rw_archie, label='rw_archie')
ax4.plot(test.DEPTH, rw_shale, label='rw_shale')
ax4.set_frame_on(False)
ax4.set_yscale('log')
ax4.legend()

ax5.plot(test.DEPTH, swt_sal, label='SWT_salinity')
ax5.plot(test.DEPTH, swt_ws, label='SWT_ws')
ax5.plot(test.DEPTH, test['SWT'], label='SWT')
ax5.set_frame_on(False)
ax5.legend()

fig.tight_layout()

In [ ]:
# Estimate permeability
perm = choo_permeability(vclw, vsilt, phit,  B=6, A=5e6)
Swirr = np.where(
    well_data['ROCK_FLAG'] == 1, 0.005, np.where(
        well_data['ROCK_FLAG'] == 2, 0.2, np.where(
            well_data['ROCK_FLAG'] == 3, 0.3, 1
        )
    )
)
perm_kc = kozeny_carman_permeability(phit, S=.5)
perm_t = timur_permeability(phit, Swirr)
perm_c = coates_permeability(phit, Swirr, a=.5)
perm_tx = tixier_permeability(phit, Swirr)

fig, ax = plt.subplots(figsize=(20, 3))
ax.plot(well_data.DEPTH, well_data['PERM'], label='Original')
# ax.scatter(well_data.DEPTH, well_data['CPERM'], label='CPERM', c='r')
ax.plot(well_data.DEPTH, perm, label='Choo')
ax.plot(well_data.DEPTH, perm_kc, label='KC')
ax.plot(well_data.DEPTH, perm_t, label='Timur')
ax.plot(well_data.DEPTH, perm_c, label='Coates')
ax.plot(well_data.DEPTH, perm_tx, label='Tixier')
ax.set_frame_on(False)
ax.set_yscale('log')
ax.set_ylim(1e-2, 1e4)
# ax.set_xlim(1000, 1200)
ax.legend(bbox_to_anchor=(1.04, 1), loc="upper left")

In [ ]:
from quick_pp.rock_type import estimate_vsh_gr
vsh_gr = estimate_vsh_gr(well_data['GR'], alpha=0.1)
fig, axs = plt.subplots(2,1, figsize=(20, 3), sharex=True)
axs[0].plot(well_data.DEPTH, vsh_gr, label='vsh_gr')
axs[1].plot(well_data.DEPTH, well_data['GR'], label='GR')
axs[0].legend()
axs[1].legend()

In [ ]:
# Density correction based on VSH_GR
rhob_corr = den_correction(well_data['NPHI'], well_data['GR'], alpha=0.1)
badholef =  np.where(abs(well_data['RHOB'] - rhob_corr) > 0.2, 1, 0)

min_depth = 1550
max_depth = 1650
plt.figure(figsize=(20, 3))
plt.plot(well_data.DEPTH, rhob_corr, label='rhob_corr')
plt.plot(well_data.DEPTH, well_data['RHOB'], label='RHOB')
plt.plot(well_data.DEPTH, well_data['RHOB_HC'], label='RHOB_HC')
plt.legend(bbox_to_anchor=(1.04, 1), loc="upper left")
plt.xlim(min_depth, max_depth)

plt.figure(figsize=(20, 3))
plt.plot(well_data.DEPTH, well_data['NPHI'], label='NPHI')
plt.plot(well_data.DEPTH, well_data['NPHI_HC'], label='NPHI_HC')
plt.legend(bbox_to_anchor=(1.04, 1), loc="upper left")
plt.xlim(min_depth, max_depth)

plt.figure(figsize=(20, 3))
plt.plot(well_data.DEPTH, vsh_gr, label='vsh_gr')
plt.legend(bbox_to_anchor=(1.04, 1), loc="upper left")
plt.xlim(min_depth, max_depth)

In [ ]:
# Plot the results
well_data.drop(columns=['VOIL', 'VGAS'], errors='ignore', inplace=True)
fig = plotly_log(well_data, depth_uom=well.depth_uom)
fig.show(config=dict(scrollZoom=True))
# fig.write_html(rf"{well_name}_log.html", config=dict(scrollZoom=True))

In [ ]:
from quick_pp.qaqc import quick_qc

test, summary_df, dist_fig, depth_fig = quick_qc(well_data)

dist_fig.show()
depth_fig.show()

In [ ]:
# # Save the results to the well
# well.update_data(well_data)
# well.update_ressum(ressum_df)
# project.save_well(well)

In [ ]:
# project.save()